In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, StringType

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
languages_schema = StructType(fields=[
    StructField("languageId", IntegerType(), True),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])

In [0]:
languages_df = spark.read \
    .option("header", "true") \
    .schema(languages_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

In [0]:
from pyspark.sql.functions import col

In [0]:
languages_selected_df = languages_df.select(col("languageId"), col("languageName"))

In [0]:
languages_renamed_df = languages_selected_df.withColumnRenamed("languageId", "language_id").withColumnRenamed("languageName", "language_name")

In [0]:
from pyspark.sql.functions import lit, current_timestamp

In [0]:
language_final_df = add_ingestion_date(languages_renamed_df).withColumn("environment", lit("production")).withColumn("file_date", lit(v_file_date))

In [0]:
language_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
%sql
SELECT * FROM movie_silver.languages

In [0]:
display(spark.read.format("delta").load(f"{silver_folder_path}/languages"))